In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
from collections import deque
from operator import itemgetter

In [2]:
df = pd.read_csv('tubedata.csv', header=None)
df.head()
# Start Station | End Station | Tube Line | Avg. Time Taken | Main Zone | Secondary Zone

,0,1,2,3,4,5
0,Harrow & Wealdstone,Kenton,Bakerloo,3,5,0
1,Kenton,South Kenton,Bakerloo,2,4,0
2,South Kenton,North Wembley,Bakerloo,2,4,0
3,North Wembley,Wembley Central,Bakerloo,2,4,0
4,Wembley Central,Stonebridge Park,Bakerloo,3,4,0


# Declaring the Dictionaries

In [3]:
'''
I HAVE TAKEN THIS CODE FROM THE COURSEWORK 1 UNDIRECTED_MAP.PY FILE

A dictionary where the values for each key are stored as lists.
Here, the key is the station name and the values are all the stations the tube lines go from the key station, the 
time taken to reach this station and the tube line needed to get there.
'''
station_dict = defaultdict(list)

'''
A dictionary where the values for each key are stored as lists.
Here, the key is the station name and the values are all the zones the station is in.
'''
zone_dict = defaultdict(set) # a dictionary where the values for each key are stored as sets

'''
A dictionary where we will store values for each zone as the zones are alphanumeric. So, the heuristic that I 
have considered cannot give me the difference between a character and number. So, I am assigning each zone a specific value 
which is a multiple of 10
'''
heu_dict = {} # a dictionary where the values for each zone are stored

In [4]:
'''
I HAVE TAKEN THIS CODE FROM THE COURSEWORK 1 UNDIRECTED_MAP.PY FILE

Run only once for optimal output as each time you run this cell, a new entry for all the stations and zones
gets added to the default dict and this can mess up the result.
'''

list_heu = ['1', '2', '3', '4', '5', '6', 'a', 'b', 'c', 'd']

'''
Iterate through the list list_heu and assign each zone a numeric value which is a multiple of 10.
So, zone 1 will have value 10, zone 2 will have value 20 ... zone d will have value 100
This will be a dictionary with values like {'1':10, '2':20, ..., 'd':100}
'''
for ind, elem in enumerate(list_heu):
    heu_dict[elem] = 10* (ind + 1)

'''
Create the Station and Zone Dictionaries.

The Station dictionary is a default dictionary of lists where we are storing the data in the form:
{'key':list[tuples]}
{'start_station':[('end_station', avg_time, 'tube line'), ('end_station', avg_time, 'tube line')]}
'''
for index, row in df.iterrows(): # iterrows() is a method provided by pandas to iterate over the rows of the DataFrame.

    start_station = row[0]
    end_station = row[1]
    tube_line = row[2]
    act_cost = int(row[3])

    zone1 = row[4]
    zone2 = row[5]

    # station dictionary of child station tuples (child_name, cost from parent to the child)
    # {"Mile End": [("Stepney Green", 2), ("Wembley", 1)]}
    station_list = station_dict[start_station]
    station_list.append((end_station, act_cost, tube_line))

    station_list = station_dict[end_station]
    station_list.append((start_station, act_cost, tube_line))

    zone_dict[start_station].add(zone1)
    if zone2 != "0":
        zone_dict[start_station].add(zone2)
        # if the secondary zone is not 0 it's the main zone for the ending station
        zone_dict[end_station].add(zone2)
    else:
        # otherwise the main zone for the ending station is the same as for the starting station
        zone_dict[end_station].add(zone1)

# print(zone_dict)

test_paths = [
    ('Euston', 'Victoria'),
    ('Canada Water', 'Stratford'),
    ('New Cross Gate', 'Stepney Green'),
    ('Ealing Broadway', 'South Kensington'),
    ('Baker Street', 'Wembley Park')
]

In [5]:
'''
I HAVE TAKEN THIS CODE FROM THE LAB 2 SOLUTION FROM THE FUNCTION "def construct_path_from_root(node, root):" UNDER THE HEADING Solution DFS -- variation 1

We call this function every time we want to calculate the path taken by the tube from the start to end station

'node' will have the solution (the path taken) by the tube in the following format for Depth First Search,
Breadth First Search and Uniform Cost Search without Line Change Implementation:
{'label':initial, 'parent':None, 'cost':0, 'line':None}
'''
def construct_path_from_root(node, root):
    # Assign the first station to the path_from_root list
    path_from_root = [node['label']]
    # Assign the first cost to the cost_from_root variable
    cost_from_root = node['cost']
    # Run the loop until node['parent'] is False
    while node['parent']:
        node = node['parent']
        # Append the next station in the solution to the path_from_root list
        path_from_root = [node['label']] + path_from_root
        # Add the cost of the next station to the cost_from_root
        cost_from_root = node['cost'] + cost_from_root

    return path_from_root, cost_from_root